In [608]:
from bs4 import BeautifulSoup
import requests
import time, os 
import pandas as pd

In [595]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

## (1) extract movies from, 2019, 2020, 2021.

In [1397]:
def extracts_fields(list_tags,year_release):

    movies_dict = {}
    for i in range(len(list_tags)): # link is a tag object now (unlike list_tags) 
        movie_name = list_tags[i].find('a').text
    
        movie_ID = list_tags[i].find('a')['href'].split("/")[2]
        imdb_movie_link = url_imdb_movie + list_tags[i].find('a')['href'].split("/")[2] + '?ref_=fn_al_tt_1'
       
        try:
            MPPA = list_tags[i].find(class_='certificate').text.strip()
        except: 
            MPPA = []
     
        try:
            runtime = list_tags[i].find(class_='runtime').text.strip().split(" ")[0]
        except:
            runtime = []
     
        try:
            genre = list_tags[i].find(class_='genre').text.strip().split(",")
        
        except:
            genre = []
        
        try: 
            imdb_rating = list_tags[i].find('strong').text.strip()
        except:
            imdb_rating = []
        
        try:
            votes = list_tags[i].find(class_='sort-num_votes-visible').find('span').findNext().text.strip().replace(',', '')
        except:
            votes = []

        try:
            metacritic = list_tags[i].find(class_='inline-block ratings-metascore').find('span').contents[0].strip()
        except: # In case there is no metacritic.
            metacritic = []
            
        if type(votes) == str:
            num_votes_last_video = int(votes)
        else:
            num_votes_last_video = votes
    
        movies_dict[movie_name] = [movie_ID]+[imdb_movie_link]+[MPPA]+[runtime]+[genre]+[imdb_rating]+\
        [votes]+[metacritic]+[year_release]
    
    return movies_dict,num_votes_last_video

In [1396]:
movies_2019.head()

,movie_ID,link,MPPA,runtime,genre,IMDB_rating,number_of_votes,metacritic,year_release
Joker,tt7286456,https://www.imdb.com/title/tt7286456?ref_=fn_a...,R,122,"[Crime, Drama, Thriller]",8.4,1067788,59,2019
Avengers: Endgame,tt4154796,https://www.imdb.com/title/tt4154796?ref_=fn_a...,PG-13,181,"[Action, Adventure, Drama]",8.4,939831,78,2019
Once Upon a Time... In Hollywood,tt7131622,https://www.imdb.com/title/tt7131622?ref_=fn_a...,R,161,"[Comedy, Drama]",7.6,637902,83,2019
Knives Out,tt8946378,https://www.imdb.com/title/tt8946378?ref_=fn_a...,PG-13,130,"[Comedy, Crime, Drama]",7.9,528298,82,2019
Captain Marvel,tt4154664,https://www.imdb.com/title/tt4154664?ref_=fn_a...,PG-13,123,"[Action, Adventure, Sci-Fi]",6.8,491971,64,2019


In [930]:
import random
from fake_useragent import UserAgent
ua = UserAgent()


In [931]:
url_imdb_movie = 'https://www.imdb.com/title/'

In [1395]:
def list_movies_year(year_release):

    imdb_query = f'https://www.imdb.com/search/title/?title_type=feature&release_date={year_release}-01-01,{year_release}-12-31&countries=us&languages=en&sort=num_votes,desc&count=250'
    user_agent = {'User-agent': ua.random} # generates a random user agent.
    page = requests.get(imdb_query,headers = user_agent).text
    soup = BeautifulSoup(page, 'html5lib')
    driver = webdriver.Chrome(chromedriver) # Web driver object.
    driver.get(imdb_query) # like navigating to this query on the browserb

    # Define headers
    headers_movies = ['movie_ID', 'link',
               'MPPA', 'runtime', 'genre','IMDB_rating','number_of_votes','metacritic','year_release']

    num_votes_last_video = 9999999 # initialize value, bigger than 1000.
    num_page = 1
    while num_votes_last_video > 200: 
        if num_page > 1: # move to the next page.
            filter_button = driver.find_element_by_xpath(
            '//a[contains(@class, "lister-page-next next-page")]')
            filter_button.click()

        soup = BeautifulSoup(driver.page_source, 'html.parser') # parse current page.
        list_tags = soup.find_all(class_="lister-item-content") # find the list tags.
        movies_dict,num_votes_last_video = extracts_fields(list_tags,year_release) # extract the movie information of interest.

        movies_current_page = pd.DataFrame(movies_dict).T
        movies_current_page.columns = headers_movies

        if num_page == 1:
            movies_list = movies_current_page
        else:
            movies_list = pd.concat([movies_list,movies_current_page],axis = 0)

        del movies_dict,movies_current_page

        time.sleep(.5+2*random.random()) # random time between requests.

        movies_list.to_csv(f'movies_list_{year_release}_until_page{num_page}.csv') # save for safety
        num_page += 1
        
    driver.close()
    
    return movies_list



In [1398]:
movies_2019 = list_movies_year(2019)
movies_2020 = list_movies_year(2020)
movies_2021 = list_movies_year(2021)

movies_2018 = list_movies_year(2018)
movies_2017 = list_movies_year(2017)
movies_2016 = list_movies_year(2016)
movies_2015 = list_movies_year(2015)


In [1403]:
movies_2015.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, Star Wars: Episode VII - The Force Awakens to Hangman's Game
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   movie_ID         1000 non-null   object
 1   link             1000 non-null   object
 2   MPPA             1000 non-null   object
 3   runtime          1000 non-null   object
 4   genre            1000 non-null   object
 5   IMDB_rating      1000 non-null   object
 6   number_of_votes  1000 non-null   object
 7   metacritic       1000 non-null   object
 8   year_release     1000 non-null   object
dtypes: object(9)
memory usage: 78.1+ KB


Concatenate the three data frames:

In [1410]:
movies_links_before_filtering = pd.concat([movies_2019,movies_2020,movies_2021,movies_2018,movies_2017,movies_2016,movies_2015],axis = 0) 
movies_links_before_filtering.info()


<class 'pandas.core.frame.DataFrame'>
Index: 6247 entries, Joker to Hangman's Game
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   movie_ID         6247 non-null   object
 1   link             6247 non-null   object
 2   MPPA             6247 non-null   object
 3   runtime          6247 non-null   object
 4   genre            6247 non-null   object
 5   IMDB_rating      6247 non-null   object
 6   number_of_votes  6247 non-null   object
 7   metacritic       6247 non-null   object
 8   year_release     6247 non-null   object
dtypes: object(9)
memory usage: 488.0+ KB


In [1412]:
movies_links_before_filtering.tail()

,movie_ID,link,MPPA,runtime,genre,IMDB_rating,number_of_votes,metacritic,year_release
The Last Apartment,tt3717824,https://www.imdb.com/title/tt3717824?ref_=fn_a...,[],83,[Horror],3.3,131,[],2015
Altar Egos,tt3647804,https://www.imdb.com/title/tt3647804?ref_=fn_a...,[],86,"[Comedy, Family]",4.3,131,[],2015
Golden Kingdom,tt3440132,https://www.imdb.com/title/tt3440132?ref_=fn_a...,Not Rated,104,[Drama],6.3,131,[],2015
Within These Walls,tt3546802,https://www.imdb.com/title/tt3546802?ref_=fn_a...,[],95,[Horror],3.2,130,[],2015
Hangman's Game,tt1827429,https://www.imdb.com/title/tt1827429?ref_=fn_a...,TV-14,87,"[Horror, Mystery, Thriller]",2.8,130,[],2015


In [1413]:
movies_links_before_filtering['number_of_votes'] = pd.to_numeric(movies_links_before_filtering['number_of_votes']) # it was a string!
movies_links_before_filtering['year_release'] = pd.to_numeric(movies_links_before_filtering['year_release']) # it was a string!

movies_links_before_filtering.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6247 entries, Joker to Hangman's Game
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   movie_ID         6247 non-null   object
 1   link             6247 non-null   object
 2   MPPA             6247 non-null   object
 3   runtime          6247 non-null   object
 4   genre            6247 non-null   object
 5   IMDB_rating      6247 non-null   object
 6   number_of_votes  6247 non-null   int64 
 7   metacritic       6247 non-null   object
 8   year_release     6247 non-null   int64 
dtypes: int64(2), object(7)
memory usage: 488.0+ KB


In [1414]:
len(movies_links_before_filtering[movies_links_before_filtering['number_of_votes'] >= 200]) # We will have 1930 pages.

5720

In [1415]:
movies_links_filtered = movies_links_before_filtering[movies_links_before_filtering['number_of_votes'] >= 200]
movies_links_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5720 entries, Joker to Richard Peter Johnson
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   movie_ID         5720 non-null   object
 1   link             5720 non-null   object
 2   MPPA             5720 non-null   object
 3   runtime          5720 non-null   object
 4   genre            5720 non-null   object
 5   IMDB_rating      5720 non-null   object
 6   number_of_votes  5720 non-null   int64 
 7   metacritic       5720 non-null   object
 8   year_release     5720 non-null   int64 
dtypes: int64(2), object(7)
memory usage: 446.9+ KB


In [1416]:
movies_links_filtered = movies_links_filtered.reset_index()
movies_links_filtered = movies_links_filtered.rename(columns = {'index':'title'})
movies_links_filtered

,title,movie_ID,link,MPPA,runtime,genre,IMDB_rating,number_of_votes,metacritic,year_release
0,Joker,tt7286456,https://www.imdb.com/title/tt7286456?ref_=fn_a...,R,122,"[Crime, Drama, Thriller]",8.4,1067788,59,2019
1,Avengers: Endgame,tt4154796,https://www.imdb.com/title/tt4154796?ref_=fn_a...,PG-13,181,"[Action, Adventure, Drama]",8.4,939831,78,2019
2,Once Upon a Time... In Hollywood,tt7131622,https://www.imdb.com/title/tt7131622?ref_=fn_a...,R,161,"[Comedy, Drama]",7.6,637902,83,2019
3,Knives Out,tt8946378,https://www.imdb.com/title/tt8946378?ref_=fn_a...,PG-13,130,"[Comedy, Crime, Drama]",7.9,528298,82,2019
4,Captain Marvel,tt4154664,https://www.imdb.com/title/tt4154664?ref_=fn_a...,PG-13,123,"[Action, Adventure, Sci-Fi]",6.8,491971,64,2019
...,...,...,...,...,...,...,...,...,...,...
5715,Keep in Touch,tt3344998,https://www.imdb.com/title/tt3344998?ref_=fn_a...,Not Rated,105,"[Comedy, Drama, Music]",6.7,204,[],2015
5716,Eyes of the Dead,tt4685626,https://www.imdb.com/title/tt4685626?ref_=fn_a...,TV-14,83,"[Action, Horror, Thriller]",3.9,203,[],2015
5717,Bigfoot the Movie,tt4637832,https://www.imdb.com/title/tt4637832?ref_=fn_a...,Not Rated,96,[Comedy],5.6,203,[],2015
5718,Alcatraz Prison Escape: Deathbed Confession,tt1379712,https://www.imdb.com/title/tt1379712?ref_=fn_a...,[],93,"[Action, Crime]",5.2,201,[],2015


In [1417]:
movies_links_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5720 entries, 0 to 5719
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            5720 non-null   object
 1   movie_ID         5720 non-null   object
 2   link             5720 non-null   object
 3   MPPA             5720 non-null   object
 4   runtime          5720 non-null   object
 5   genre            5720 non-null   object
 6   IMDB_rating      5720 non-null   object
 7   number_of_votes  5720 non-null   int64 
 8   metacritic       5720 non-null   object
 9   year_release     5720 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 447.0+ KB


In [1425]:
movies_links_filtered.loc[0]

title                                                          Joker
movie_ID                                                   tt7286456
link               https://www.imdb.com/title/tt7286456?ref_=fn_a...
MPPA                                                               R
runtime                                                          122
genre                                     [Crime,  Drama,  Thriller]
IMDB_rating                                                      8.4
number_of_votes                                              1067788
metacritic                                                        59
year_release                                                    2019
Name: 0, dtype: object

# And now, web-scrapping on each movie page:

In [1351]:
def total_awards_extractor(condition):
    awards_info = soup.find('span',class_='ipc-metadata-list-item__list-content-item').text
    if condition in awards_info:
        for type_award in awards_info.split('&'): # split to separate between wins and nominations.
            if condition in type_award:
                return int(type_award.strip().split(' ')[0])
        

In [1437]:
def scraps_movie_ID_page(soup,title,movie_ID,index_movie):

    movie_page = {} 
    
    num_user_reviews = 0
    num_critic_reviews = 0
    total_awards_wins = 0
    total_awards_nominations = 0
    oscar_wins = 0
    oscar_nominations = 0

    # Release date and Country:
    try:
        release_info = soup.find(text=re.compile('Release date')).findNext().text.split('(')
        release_date = release_info[0].strip() 
        release_country = release_info[1].split(')')[0]

    except:
        release_date = []
        release_country = []

    # number user reviews: 
    try:
        for i in range(len(soup.find_all(class_="label"))):
            if soup.find_all(class_="label")[i].text == 'User reviews':
                num_user_reviews = soup.find_all(class_="label")[i].findPrevious().text
    except:  
        num_user_reviews = 0

    # Number of critic reviews:    
    try:
        num_critic_reviews = soup.find(text=re.compile('Critic reviews')).findPrevious().findPrevious().text
    except: 
        num_critic_reviews = 0


    # Total awards and nominations:
    try:
        total_awards_wins = total_awards_extractor('win')
    except:  
        total_awards_wins = 0
    try:
        total_awards_nominations = total_awards_extractor('nomination')
    except:
        total_awards_nominations = 0


    # Total Oscar Wins:
    try:
        span_candidates = soup.find_all('a',class_="ipc-metadata-list-item__label ipc-metadata-list-item__label--link")
        for span_candidate in span_candidates:
            if "Won" in span_candidate.text: # or "Nominated" in span_candidate.text
                oscar_wins = span_candidate.text.split(' ')[1]
    except:
        oscar_wins = 0
    
    # Total Oscar Nominations (only reported if there were no wins):
    try:
        span_candidates = soup.find_all('a',class_="ipc-metadata-list-item__label ipc-metadata-list-item__label--link")
        for span_candidate in span_candidates:
            if "Nominated" in span_candidate.text: 
                oscar_nominations = span_candidate.text.split(' ')[2]     
    except:
        oscar_nominations = 0    

    # Production companies:
    try:
        classes_repetitions = soup.find_all(class_="ipc-metadata-list-item__label ipc-metadata-list-item__label--link")
        production_companies = []
        for class_repetition in classes_repetitions:
            if class_repetition.text == 'Production companies' or class_repetition.text ==  'Production company':
                companies = class_repetition.findNext().find_all(class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")
                for current_company in companies:
                    if current_company.text not in production_companies:
                        production_companies.append(current_company.text)  
    except:  
        production_companies = []

    # Box office: 
    try:
        budget = soup.find(text=re.compile('Budget')).findNext().text.replace(',','').split(' ')[0]
    except:
        budget = []
    try:    
        opening_weekend_usa_can = soup.find(text=re.compile('Opening weekend US & Canada')).findNext().findNext().\
            findNext().text.replace(',','')
    except:
        opening_weekend_usa_can = []

    try:
        gross_usa_can = soup.find(text=re.compile('Gross US & Canada')).findNext().findNext().findNext().text.replace(',','')
    except:    
        gross_usa_can = []

    try:    
        gross_worldwide = soup.find(text=re.compile('Gross worldwide')).findNext().text.replace(',','')
    except:
        gross_worldwide = []

    # Director
    try:
        director = soup.find(text=re.compile('Director')).findNext().findNext().findNext().text
    except: 
        director = []
    # Top actor:
    try:
        top_star = soup.find(text=re.compile('Stars')).findNext().findNext().findNext().findNext().text
    except:   
        top_star = []        
    # Top writer:
    try:
        top_writer = soup.find(text=re.compile('Writer')).findNext().findNext().findNext().findNext().text
    except:   
        top_writer = []
         
    movie_page[index_movie] = [title]+[movie_ID]+[release_date]+[release_country]+[num_user_reviews]+[num_critic_reviews]+\
    [total_awards_wins]+[total_awards_nominations]+[oscar_wins]+[oscar_nominations]+[production_companies]+[budget]+[opening_weekend_usa_can]+\
    [gross_usa_can]+[gross_worldwide]+[director]+[top_star]+[top_writer]
    
    
    time.sleep(.5+2*random.random()) # random time between requests.
    

    return movie_page 

In [1426]:
len(movies_links_filtered)

5720

In [ ]:
# Define headers
headers_movies = ['title','movie_ID','release_date','release_country','num_user_reviews','num_critic_reviews',\
                  'total_awards_wins','total_nominations',
                  'oscar_wins','oscar_nominations','production_companies','budget','opening_weekend_usa_can',\
                  'gross_usa_can','gross_worldwide', 'director','top_star','top_writer']

for i in range(len(movies_links_filtered)):
    
    link_imdb = movies_links_filtered['link'][i]        
    user_agent = {'User-agent': ua.random} # generates a random user agent.
    page = requests.get(link_imdb,headers = user_agent).text
    soup = BeautifulSoup(page, 'html5lib')
    driver = webdriver.Chrome(chromedriver) # Web driver object.
    driver.get(link_imdb) # like navigating to this query on the browser


    soup = BeautifulSoup(driver.page_source, 'html.parser') # parse current page.
    current_movie_dict = scraps_movie_ID_page(soup,movies_links_filtered['title'][i],movies_links_filtered['movie_ID'][i],i)
        
    current_movie_df = pd.DataFrame(current_movie_dict).T
    current_movie_df.columns = headers_movies

    if i == 0:
        movies_df = current_movie_df
    else:
        movies_df = pd.concat([movies_df,current_movie_df],axis = 0)

    del current_movie_dict,current_movie_df  
        
        
    if ((i+1) % 10 == 0):
        movies_df.to_csv(f'movies_database_until_movie_{i}.csv') # save for safety               
        
    print(i)
        
    driver.close()
        
    if ((i+1) % 100 == 0):
        time.sleep(320)
            


0
1


In [1392]:
movie_df.info()

2249